In [44]:
# Imports

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import requests
from datetime import datetime, timedelta

In [61]:
# La fonction renvoie sous forme de dataframe, le taux d'occupation et le débit pour chaque heure de la journée


def get_road_data_by_date(year, month, day, libelle, libelle_nd_amont, libelle_nd_aval):

    # API Url
    api_url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/comptages-routiers-permanents/records"

    # Request parameters
    params = {
        "limit": 24,
        "order_by": "t_1h",
        "timezone": "CET",
        "refine": [
            f"t_1h:{year}/{month}/{day}",
            f"libelle_nd_amont:{libelle_nd_amont}",
            f"libelle_nd_aval:{libelle_nd_aval}",
            f"libelle:{libelle}"
        ]
    }

    # Get response
    response = requests.get(api_url, params=params)

    # Fetch data
    if response.status_code == 200:
        records = response.json().get("results", [])
        if records:
            df = pd.DataFrame(columns=["year", "month", "day", "hour", "q", "k"])
            for index, record in enumerate(records):
                t_1h = record["t_1h"]
                q = record["q"]
                k = record["k"]
                year = t_1h[:4]
                month = t_1h[5:7]
                day = t_1h[8:10]
                hour = t_1h[11:13]
                
                df.loc[index] = [year, month, day, hour, q, k]
        else:
            print("Aucune donnée trouvée pour les critères spécifiés.")
    else:
        print(f"API Error : {response.status_code}")

    return df


In [64]:
print(get_road_data_by_date("2023", "12", "06", "AV_Champs_Elysees", "Av_Champs_Elysees-Washington", "Av_Champs_Elysees-Berri"))
print(get_road_data_by_date("2023", "12", "06", "Convention", "Convention-Blomet", "Lecourbe-Convention"))
print(get_road_data_by_date("2023", "12", "06", "St_Antoine", "Bastille-St_Antoine", "St_Antoine-Jacques_Coeur"))

# q: h flow rate (number of vehicle per hour)
# k: occupancy rate (time spent by a vehicle in one hour, in %, 100% = 1 hour)

    year month day hour       q         k
0   2023    12  06   00   684.0  11.75333
1   2023    12  06   01   624.0   8.16334
2   2023    12  06   02   478.0   5.59167
3   2023    12  06   03   664.0  14.25889
4   2023    12  06   04   418.0   9.83778
5   2023    12  06   05   261.0   3.38778
6   2023    12  06   06   442.0   5.95889
7   2023    12  06   07   777.0  10.27667
8   2023    12  06   08  1083.0  11.97778
9   2023    12  06   09  1408.0  19.71333
10  2023    12  06   10  1523.0  22.98500
11  2023    12  06   11  1400.0  23.02445
12  2023    12  06   12  1391.0  23.07111
13  2023    12  06   13  1346.0  28.98111
14  2023    12  06   14  1139.0  27.49500
15  2023    12  06   15  1214.0  23.64222
16  2023    12  06   16  1210.0  23.14000
17  2023    12  06   17  1258.0  29.99556
18  2023    12  06   18  1236.0  31.21167
19  2023    12  06   19  1456.0  33.67167
20  2023    12  06   20  1395.0  26.13722
21  2023    12  06   21  1206.0  23.97111
22  2023    12  06   22  1106.0  1